In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from data_process import tokenizers, utils
from nanoGPT.model import  GPT, GPTConfig
from lightning_training import LightningGPT, GamesDataModule, GamesDataset
import pytorch_lightning as pl
import torch
import os
import numpy as np
import seaborn as sns
from data_process.utils import add_elo_token_to_games, join_material_tokens, remove_material_tokens

In [2]:
headers = ["index", "id", "date", "white_elo", "black_elo", "result", "ply", "ply_30s", "piece_uci"]

games_df = pd.read_csv("./data/test.csv", delimiter=";", header=None, names=headers)

# games_df = games_df.sample(frac=0.1, random_state=42)

games_df

,index,id,date,white_elo,black_elo,result,ply,ply_30s,piece_uci
0,74497,?,2021.12.05,2523,2684,0-1,52,-1,Pd2d4 39 39 Pd7d5 39 39 Nb1c3 39 39 Pc7c6 39 3...
1,74505,?,2021.12.05,2623,2581,0-1,96,-1,Pd2d4 39 39 Ng8f6 39 39 Pc2c4 39 39 Pe7e6 39 3...
2,74508,?,2021.12.05,2757,2670,1/2-1/2,106,-1,Pd2d4 39 39 Pd7d5 39 39 Ke1d2 39 39 Pe7e5 39 3...
3,74510,?,2021.12.05,2627,2666,0-1,96,-1,Pd2d4 39 39 Ng8f6 39 39 Bc1f4 39 39 Pd7d5 39 3...
4,74517,?,2021.12.05,2666,2571,1-0,47,-1,Pe2e4 39 39 Pc7c5 39 39 Ng1f3 39 39 Nb8c6 39 3...
...,...,...,...,...,...,...,...,...,...
99995,14202188,CJayVtXh,2024.12.05,1303,879,1-0,33,-1,Pe2e4 39 39 Pe7e6 39 39 Ng1f3 39 39 Pf7f6 39 3...
99996,14202189,XkroYjvj,2024.12.05,1099,1209,0-1,59,-1,Pe2e4 39 39 Pe7e5 39 39 Ng1f3 39 39 Nb8c6 39 3...
99997,14202191,DWLSh6Wn,2024.12.05,1070,1019,0-1,48,-1,Pe2e4 39 39 Pe7e5 39 39 Ng1f3 39 39 Ng8f6 39 3...
99998,14202194,g7tUv4Rt,2024.12.05,1211,1173,1-0,9,-1,Pe2e4 39 39 Pd7d5 39 39 Bf1c4 39 39 Pe7e5 39 3...


In [3]:
games = join_material_tokens(games_df.piece_uci, replace_bigger_values=True)
games = add_elo_token_to_games(games, games_df.white_elo, games_df.black_elo)

In [4]:
tokenizer = tokenizers.FullMoveEloMaterialPairTokenizer()

model_config = GPTConfig(
    block_size=604,
    vocab_size=tokenizer.vocab_size,
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

test_start_token = 21
test_token_step = 2


material_checkpoint = "./models/small_training/elo_with_material_pair/epoch=4-step=78125.ckpt"

material_model = LightningGPT.load_from_checkpoint(
    material_checkpoint,
    config=model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
)

material_checkpoint_2 = "./models/small_training/elo_material_pair_ignore_material_prediction/epoch=4-step=78125.ckpt"
material_model_2 = LightningGPT.load_from_checkpoint(
    material_checkpoint_2,
    config=model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
)

base_checkpoint = "./models/small_training/elo_base/epoch=4-step=78125.ckpt"

base_model = LightningGPT.load_from_checkpoint(
    base_checkpoint,
    config=model_config,
    test_start_token=11,
    test_token_step=1,
)

material_checkpoint_3 = "./models/full_training/elo_material_pair_ignore_material_prediction/epoch=9-step=1250000.ckpt"
material_model_3 = LightningGPT.load_from_checkpoint(
    material_checkpoint_3,
    config=model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
)

number of parameters: 28.15M
number of parameters: 28.15M
number of parameters: 28.15M
number of parameters: 28.15M


In [5]:
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=10,
    callbacks=[pl.callbacks.RichProgressBar()],
    precision="bf16-mixed",
)
torch.set_float32_matmul_precision('high')

Using bfloat16 Automatic Mixed Precision (AMP)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
data_module = GamesDataModule(test_games=list(games), tokenizer=tokenizer, num_workers=12)

In [8]:
trainer.test(material_model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.40388044714927673    │
│         test_loss         │    1.1656888723373413     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.1656888723373413, 'test_acc': 0.40388044714927673}]

In [7]:
trainer.test(material_model_2, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.41936370730400085    │
│         test_loss         │     9.046186447143555     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 9.046186447143555, 'test_acc': 0.41936370730400085}]

In [7]:
trainer.test(material_model_3, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5091182589530945     │
│         test_loss         │    0.9533836245536804     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.9533836245536804, 'test_acc': 0.5091182589530945}]

In [17]:
base_games = remove_material_tokens(games_df.piece_uci)
base_games = add_elo_token_to_games(base_games, games_df.white_elo, games_df.black_elo)
base_games

75721    2400 2400 Pd2d4 Ng8f6 Ng1f3 Pc7c5 Pe2e3 Pb7b6 ...
80184    1900 2000 Pd2d4 Ng8f6 Nb1c3 Pc7c5 Pd4d5 Pd7d6 ...
19864    2100 2100 Ng1f3 Pc7c6 Pg2g3 Pd7d5 Pd2d3 Pe7e6 ...
76699    2400 2400 Pc2c4 Pe7e6 Nb1c3 Pd7d5 Pd2d4 Pc7c5 ...
92991    1200 1100 Pe2e4 Pd7d5 Pe4d5 Qd8d5 Nb1c3 Qd5d8 ...
                               ...                        
5002     2700 2500 Pd2d4 Pd7d5 Nb1c3 Ng8f6 Bc1f4 Pc7c5 ...
30151    2200 2100 Pc2c4 Pe7e6 Ng1f3 Pd7d5 Pg2g3 Ng8f6 ...
93194    1000 1100 Pe2e4 Pe7e5 Pd2d3 Nb8c6 Nb1c3 Ng8f6 ...
73199    2400 2400 Pe2e4 Pd7d5 Pe4d5 Qd8d5 Nb1c3 Qd5d6 ...
74479    2500 2500 Pd2d4 Ng8f6 Nb1c3 Pg7g6 Ng1f3 Pd7d5 ...
Length: 10000, dtype: object

In [19]:
base_data_module = GamesDataModule(test_games=list(base_games), tokenizer=tokenizer, num_workers=12)

In [ ]:
# model trained without material tokens is better
trainer.test(base_model, datamodule=base_data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.4135570526123047     │
│         test_loss         │    2.1594367027282715     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.1594367027282715, 'test_acc': 0.4135570526123047}]